In [1]:
import numpy as np
import math
import pandas as pd
from gplearn.genetic import SymbolicRegressor
from sklearn.model_selection import train_test_split
from sympy import *
from sklearn import metrics 
from gplearn.functions import make_function

In [2]:
df_train = pd.read_csv('tmlcc-2021/train.csv')
df_test = pd.read_csv('tmlcc-2021/test.csv')
df_pre_train = pd.read_csv('tmlcc-2021/pretest.csv')

In [3]:
df_train = df_train.iloc[:, [1, 2, 3, 4, 5, 11, 12, 13]]

In [4]:
df_train.columns = ['a','b','c','d','e','f','g', 'label']
df_train.head()

,a,b,c,d,e,f,g,label
0,1116.667429,875.240600,0.00,0.07899,0.0607,22.864166,6.786041,105.284502
1,2769.503842,2211.697211,603.61,0.13794,0.1040,33.616780,7.147286,101.224774
2,1089.818728,773.687960,788.50,0.14874,0.1262,19.263726,6.347967,118.987011
3,2205.198301,1304.638720,1441.53,0.21814,0.2220,25.701377,6.190085,187.626004
4,1137.800963,901.736120,0.00,0.07778,0.0591,30.001838,6.478063,79.210001


In [5]:
df_train.isna().sum(axis=0)

a           0
b           0
c           0
d           0
e           0
f           0
g        2087
label       0
dtype: int64

In [6]:
# Max_heat = df_train['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].replace([np.inf], 0).max()
# df_train['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].replace([np.inf], Max_heat, inplace=True)
# df_train['heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]'].fillna(df_train["heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]"].median(), inplace=True)
df_train.replace([np.inf], np.nan, inplace=True)
df_train = df_train.dropna()

In [7]:
y = df_train.pop("label")
X = df_train                                                                           
y_true = y
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
converter = {
    'sub': lambda x, y : x - y,
    'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'sin': lambda x    : sin(x),
    'cos': lambda x    : cos(x),
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
}

In [9]:
# est_gp.fit(X_train, y_train)
# print('R2:',est_gp.score(X_test,y_test))

In [10]:
# next_e = sympify((est_gp._program), locals=converter)
# next_e

In [11]:
# y_gp = est_gp.predict(X_test)
# score_gp = est_gp.score(X_test, y_test)

In [15]:
def pow_3(x1):
    f = x1**3
    return f


pow_3 = make_function(function=pow_3,name='pow3',arity=1)
# add the new function to the function_set

function_set = ['add', 'sub', 'mul', 'div', 'neg','inv','log', pow_3]
est_gp = SymbolicRegressor(population_size=1000,function_set=function_set,
                           generations=45, stopping_criteria=0.01,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=0,
                          feature_names=X.columns)
est_gp.fit(X, y)
print('R2:',est_gp.score(X, y))

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.29     9.52073e+136       25          63.6051          64.3579     13.01m
   1     8.71      2.66357e+46        6          47.5595          47.6078      8.50m
   2    10.02     4.76037e+136       26          45.8721          46.7225      8.40m
   3    11.97       2.8913e+41       11          43.8354          42.9779      7.31m
   4    12.27     1.08093e+110       17          43.2813          44.0629      7.15m
   5    17.32     4.41678e+136       26          40.5157          41.0326      6.67m
   6    18.41      2.40352e+40       26          40.4477          41.6452      6.72m
   7    18.19      2.32785e+32       25          40.3952          42.1177      7.96m
   8    25.33      2.66343e+46       40          39.4427          40.1364  

In [16]:
print(est_gp._program)

sub(mul(log(g), add(inv(neg(e)), add(inv(neg(e)), add(neg(g), sub(mul(g, g), neg(pow3(log(pow3(log(add(pow3(log(f)), d))))))))))), log(a))


In [17]:
next_e = sympify(sympify(est_gp._program), locals=converter)
next_e

sub(mul(log(g), add(inv(neg(e)), add(inv(neg(e)), add(neg(g), sub(mul(g, g), neg(pow3(log(pow3(log(add(pow3(log(f)), d))))))))))), log(a))

In [18]:
y_pred = est_gp.predict(X)

In [19]:
mse = np.log(metrics.mean_absolute_error(y, y_pred))
print("log Mean absolute Error {}".format(mse))

log Mean absolute Error 3.5237170610716606


In [21]:
df_test = df_test.iloc[:, [1, 2, 3, 4, 5, 11, 12]]
df_test.columns = ['a','b','c','d','e','f','g']

In [24]:
pred = est_gp.predict(df_test)
ID = np.arange(68614, 85614)
pd.DataFrame({'id':ID, 'CO2_working_capacity [mL/g]': pred}).set_index('id').to_csv(f'wonderland_GA1.csv')